# Rest API - Advanced Hunting Query

<table align="left">
  <tr>
    <th>Type</th>
      <th>Title</th>
    <th>Description</th>
  </tr>
  <tr>
      <td>Documentation</td>
    <td><b><a href="https://docs.microsoft.com/en-us/windows/security/threat-protection/microsoft-defender-atp/run-advanced-query-api">Advanced Hunting API</a></td>
    <td>This is the documentation that refers to what permissions is required and request examples you can use to query the MDATP API </td>
  </tr>
</table>

## Pre-requisite


## Replace variables with pertaining to your environment

In [40]:
############ This is the credentials used to authenticate with the Graph API ###################
$credentials = @{
    tenantDomain = "domain.com"
    tenantId = "xxxxxxx-xxxx-xxxx-xxxx-xxxxxxxx"        ## AAD Tenant ID
    client_id = "ae4fa870-23af-4ae5-a6ed-xxxxx"     ## Application Client ID
    client_secret = "x~xxxxxxx~xxxxx~xxxxxxxxx-xxx"   ## Application Client Secret
}

## Import required modules and credentials 

In [1]:
$modules = @('pp-core','graph') ## These are the modules to import 
$credentialPath = "~/.credentials/graph.credential" ## Where to store credentials

################## Import Graph Authentication Module #############################
ForEach ($module in $modules) {
    Import-Module "~/Notebooks/Powershell-Playground/PowerShell/custom-modules/$module" -Force  ## import module 
    Get-Command -Module "$module"                                                        ## list commands in this module 
}

$credentials = Get-StoredCredentials -credentialPath $credentialPath

The below script needs to be able to find the current output cell; this is an easy method to get it.


CommandType     Name                                               Version    Source
-----------     ----                                               -------    ------
Function        Get-DecryptedStringUsingMasterPassword             0.0        pp-core
Function        Get-EncryptedStringUsingMasterPassword             0.0        pp-core
Function        Get-HashOfString                                   0.0        pp-core
Function        Get-MasterPassword                                 0.0        pp-core
Function        Get-StoredCredentials                              0.0        pp-core
Function        Invoke-CheckCredentials                            0.0        pp-core
Function        New-StoreCredentials                               0.0        pp-core
Function        Get-MSGraphAuthToken                               0.0        graph
Function        Invoke-GraphAuthentication                         0.0        graph
Function        Invoke-MSGraphQuery                        

 ·········


## Authenticate with Graph API and get access Token for MDATP Resource

In [2]:
## Get a token if authenticates properly.
$authParams = @{
    tenantId = $credentials.tenantId
    client_id = $credentials.client_id
    client_secret = $credentials.client_secret
    resource = "https://api.securitycenter.windows.com"    ## resource Dont change since we want to query MDATP REST API Resource
    grant_type = "client_credentials"  ## This is using a appliation ID and secret to authenticate
}; $authParams

$tokenResponse = Invoke-GraphAuthentication -authParams $authParams
$tokenResponse


Name                           Value
----                           -----
resource                       https://api.securitycenter.windows.com
client_id                      ae4fa870-23af-4ae5-a6ed-5ab1811858c7
tenantId                       de40cf7e-ad5f-4245-a317-14be39cbb0ef
grant_type                     client_credentials
client_secret                  5~Q5En6ecx9d~ZtuqPf~YRBLw5S858-7WS

Authentication Parameters detected
Personal Access Token (PAT) grant_type

----------------------------------------------------------------------------
Authentiating with Microsoft Graph API using a Personal Access Token (PAT)
https://docs.microsoft.com/en-us/azure/storage/common/storage-auth-aad-app
----------------------------------------------------------------------------
Requesting Token at https://login.microsoftonline.com/de40cf7e-ad5f-4245-a317-14be39cbb0ef/oauth2/token


Received Token!
Connected and Access Token received and will expire 

token_type     : Bearer
expires_in     : 3599
e

### Put your Hunting Query here

In [3]:
## Enter your query here
$Query = "
DeviceNetworkEvents 
| where InitiatingProcessFileName =~ 'powershell.exe'
| limit 10
"

## This is the query converted to JSON format
$body = @{Query = $Query} | ConvertTo-JSON
Write-Host -ForeGround Yellow "`nQuery"; $Query
Write-Host -ForeGround Yellow "JSON"; $body


Query

DeviceNetworkEvents 
| where InitiatingProcessFileName =~ 'powershell.exe'
| limit 10

JSON
{
  "Query": "\nDeviceNetworkEvents \n| where InitiatingProcessFileName =~ 'powershell.exe'\n| limit 10\n"
}


## Query the MDATP API

### Query the MDATP API Grab report schema 

In [6]:
$URI = "https://api.securitycenter.windows.com/api/advancedqueries/run"   ## This is the URL to query with the access token.
$header = @{                                                              ## This is the header
    'Content-Type' = 'application/json'
    Accept = 'application/json'
    Authorization = "Bearer $($tokenResponse.access_token)" }

$Result = (Invoke-RestMethod -Method POST -Uri $URI -Headers $header -body $body -ErrorAction Stop)
$Result


Stats                                                                           Schema
-----                                                                           ------             
@{ExecutionTime=0.0469037; resource_usage=; dataset_statistics=System.Object[]} {@{Name=Timestamp;…



### Parse report

In [7]:
$includeColumns = @('Timestamp','DeviceName','ActionType','RemoteIP')
$Result.Results | Select-Object -Property $includeColumns


Timestamp            DeviceName ActionType        RemoteIP
---------            ---------- ----------        --------
6/12/2020 5:52:00 PM jing-lab   ConnectionSuccess 116.202.55.106

